In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
import os
import matplotlib.pyplot as plt
from scipy.stats import norm
from copy import deepcopy
from sklearn import tree
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_absolute_percentage_error
import warnings
import pickle
from pprint import pprint
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import seaborn as sns 
from sklearn.model_selection import train_test_split
import random

from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from scipy.stats import norm
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import re
import time
import sys
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
plt.style.use('ggplot')

warnings.filterwarnings("ignore")

from sklearn.datasets import make_classification

%matplotlib inline
plt.style.use('ggplot')

# Import MLflow and DAGsHub
import mlflow
import mlflow.sklearn
import dagshub

In [4]:
# Initialize DAGsHub and MLflow
dagshub.init(url="https://github.com/JuanPab2009/ProyectoFinalCD.git", mlflow=True)
MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()
print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="LaLiga-prediction")

Accessing as JuanPab2009

Initialized MLflow to track repo "JuanPab2009/ProyectoFinalCD"

Repository JuanPab2009/ProyectoFinalCD initialized!

https://github.com/JuanPab2009/ProyectoFinalCD.mlflow


MlflowException: API request to endpoint /api/2.0/mlflow/experiments/get-by-name failed with error code 404 != 200. Response body: 'Not Found'

In [ ]:
df = pd.read_excel('LaLiga Dataset 2023-2024.xlsx')
df.tail()

In [ ]:
df.shape

In [4]:
X=df[['Día','Sedes','Edad(opp)','Pos.(opp)', 'Ass(opp)', 'TPint(opp)', 
      'PrgC(opp)', 'PrgP(opp)','% de TT(opp)', 'Dist(opp)', '% Cmp(opp)', 'Dist. tot.(opp)','TklG(opp)', 'Int(opp)', 
      'Err(opp)', 'RL(opp)', 'PG(opp)', 'PE(opp)','PP(opp)', 'GF(opp)', 'GC(opp)', 'xG(opp)', 'xGA(opp)','Últimos 5(opp)', 
      'Máximo Goleador del Equipo(opp)', 'Edad(tm)', 'Pos.(tm)', 'Ass(tm)', 'TPint(tm)', 'PrgC(tm)', 'PrgP(tm)',
      '% de TT(tm)', 'Dist(tm)', '% Cmp(tm)', 'Dist. tot.(tm)', 'TklG(tm)','Int(tm)', 'Err(tm)', 'RL(tm)', 'PG(tm)', 
      'PE(tm)', 'PP(tm)', 'GF(tm)','GC(tm)', 'xG(tm)', 'xGA(tm)', 'Últimos 5(tm)','Máximo Goleador del Equipo(tm)']]
y=df['Resultado']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

### Regresión logística sin parámetros con MLflow tracking

In [ ]:
with mlflow.start_run(run_name="Logistic Regression"):
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    acc = logreg.score(X_test, y_test)
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_params(logreg.get_params())
    mlflow.log_metric("accuracy", acc)
    # Confusion matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(4, 4))
    sns.heatmap(cm, annot=True, fmt='.0f')
    plt.xlabel("Predicted Digits")
    plt.ylabel("True Digits")
    plt.title('Logistic Regression')
    plt.show()
    # Save confusion matrix plot
    plt.savefig("confusion_matrix_lr.png")
    mlflow.log_artifact("confusion_matrix_lr.png")
    # Log the model
    mlflow.sklearn.log_model(logreg, "model")
    # Get run ID
    lr_run_id = mlflow.active_run().info.run_id

### Sobremuestreo y escalamiento

In [10]:
X_train_res, y_train_res = X_train, y_train

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [12]:
sc = StandardScaler()
X_train_res = sc.fit_transform(X_train_res)
X_test = sc.transform(X_test)

In [13]:
n_folds = 5
seed = 7
kfold = KFold(n_splits=n_folds, random_state=seed, shuffle=True)

### Regresión logística con validación cruzada y MLflow

In [ ]:
with mlflow.start_run(run_name="Logistic Regression with CV"):
    # crear modelo
    log_model = LogisticRegression()
    #Hacer el cross validation y probar con el train
    log_acc = cross_val_score(log_model, X_train_res, y_train_res, scoring='accuracy', cv=kfold)
    log_prec = cross_val_score(log_model, X_train_res, y_train_res, scoring='precision_weighted', cv=kfold)
    log_rec = cross_val_score(log_model, X_train_res, y_train_res, scoring='recall_weighted', cv=kfold)

    # Performance
    print('Accuracy: %.4f' % (np.mean(log_acc)))
    print('Precision: %.4f' % (np.mean(log_prec)))
    print('Recall: %.4f' % (np.mean(log_rec)))

    # Entrenar modelo
    log_model.fit(X_train_res, y_train_res)
    y_pred = log_model.predict(X_test)

    acc_lr=accuracy_score(y_test,y_pred)
    prec_lr=precision_score(y_test,y_pred,average='weighted')
    rec_lr=recall_score(y_test,y_pred,average='weighted')

    # Log parameters
    mlflow.log_param("model_type", "LogisticRegression_CV")
    mlflow.log_params(log_model.get_params())
    # Log metrics
    mlflow.log_metric("accuracy", acc_lr)
    mlflow.log_metric("precision", prec_lr)
    mlflow.log_metric("recall", rec_lr)
    # Log model
    mlflow.sklearn.log_model(log_model, "model")
    # Get run ID
    lr_cv_run_id = mlflow.active_run().info.run_id

# SVC con MLflow tracking

In [ ]:
with mlflow.start_run(run_name="SVC"):
    from sklearn.decomposition import PCA
    from sklearn.svm import SVC
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.metrics import accuracy_score, precision_score, recall_score
    from scipy.stats import uniform

    # PCA
    pca = PCA(n_components=X_train_res.shape[1], whiten=True, random_state=42)
    X_train_pca = pca.fit_transform(X_train_res)
    X_test_pca = pca.transform(X_test)

    # Inicializar modelo SVC
    svc = SVC(kernel='rbf', class_weight='balanced')

    # Definir los hiperparámetros para la búsqueda aleatoria
    param_dist = {
        'C': uniform(1, 50),  # Distribución uniforme de valores entre 1 y 50
        'gamma': uniform(0.0001, 0.01)
    }

    # RandomizedSearchCV con búsqueda aleatoria
    grid = RandomizedSearchCV(
        svc,
        param_distributions=param_dist,
        n_iter=20,  # Realiza 20 búsquedas aleatorias
        cv=3,  # Validación cruzada con 3 folds
        n_jobs=-1,  # Utiliza todos los núcleos
        random_state=42
    )

    # Entrenar el modelo
    grid.fit(X_train_pca, y_train_res)

    # Imprimir los mejores hiperparámetros encontrados
    print("Mejores parámetros encontrados:", grid.best_params_)

    # Creación del modelo con los parámetros óptimos
    model_new_svc = grid.best_estimator_

    # Predecir
    yfit_svc = model_new_svc.predict(X_test_pca)

    # Evaluación del modelo
    yhat_svc = model_new_svc.predict(X_test_pca)
    accu_svc = accuracy_score(y_test, yhat_svc)
    prec_svc = precision_score(y_test, yhat_svc, average='weighted')
    reca_svc = recall_score(y_test, yhat_svc, average='weighted')

    print(f'Accuracy: {accu_svc:.3f}')
    print(f'Precision: {prec_svc:.3f}')
    print(f'Recall: {reca_svc:.3f}')

    # Log parameters
    mlflow.log_param("model_type", "SVC")
    mlflow.log_params(grid.best_params_)
    # Log metrics
    mlflow.log_metric("accuracy", accu_svc)
    mlflow.log_metric("precision", prec_svc)
    mlflow.log_metric("recall", reca_svc)
    # Log model
    mlflow.sklearn.log_model(model_new_svc, "model")
    # Get run ID
    svc_run_id = mlflow.active_run().info.run_id

# RandomForest con MLflow tracking

In [ ]:
with mlflow.start_run(run_name="RandomForest"):
    #Usando cross validation y grid search
    from sklearn.model_selection import GridSearchCV

    model = RandomForestClassifier(criterion='gini',
                                   min_samples_leaf=2,
                                   bootstrap=True,
                                   oob_score=False,
                                   random_state=42,
                                   verbose=2)


    #Grid search para optimizar hiperparámetros

    gs = GridSearchCV(model,
                      param_grid = {'max_depth': range(1, 11), #profundidad del árbol
                                    'min_samples_split': range(1, 10, 2),
                                    'n_estimators': range(1,15,2) #número de árboles
                                    }, 
                      cv=kfold,
                      scoring='accuracy'
                      )


    #entrenar modelo
    gs.fit(X_train_res, y_train_res)


    #imprimir hiperparámetros óptimos
    print(gs.best_params_)

    new_model = RandomForestClassifier(n_estimators=gs.best_params_['n_estimators'],
                                   criterion='gini',
                                   max_depth=gs.best_params_['max_depth'],
                                   min_samples_split=gs.best_params_['min_samples_split'],
                                   min_samples_leaf=2,
                                   bootstrap=True,
                                   oob_score=False,
                                   random_state=42,
                                   verbose=2)

    #entrenar nuevo modelo
    new_model.fit(X_train_res, y_train_res)

    yhat_rf = new_model.predict(X_test)
    accu_rf = accuracy_score(y_test,yhat_rf)
    prec_rf = precision_score(y_test,yhat_rf,average='weighted')
    reca_rf = recall_score(y_test,yhat_rf,average='weighted')
    print('Accuracy\t Precision\t Recall\n %0.3f\t %0.3f\t %0.3f'%(accu_rf,prec_rf,reca_rf))

    # Log parameters
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_params(gs.best_params_)
    # Log metrics
    mlflow.log_metric("accuracy", accu_rf)
    mlflow.log_metric("precision", prec_rf)
    mlflow.log_metric("recall", reca_rf)
    # Log model
    mlflow.sklearn.log_model(new_model, "model")
    # Get run ID
    rf_run_id = mlflow.active_run().info.run_id

### XGBoost con MLflow tracking

In [ ]:
with mlflow.start_run(run_name="XGBoost"):
    import xgboost as xgb
    from sklearn.metrics import accuracy_score, precision_score, recall_score

    # Prepare the data in DMatrix format
    dtrain = xgb.DMatrix(X_train_res, label=y_train_res-1)
    dtest = xgb.DMatrix(X_test, label=y_test-1)

    # Set parameters
    params = {
        'objective': 'multi:softprob',
        'num_class': 3,
        'gamma': 1,
        'learning_rate': 0.1,
        'max_depth': 10,
        'reg_lambda': 1,
        'scale_pos_weight': 1,
        'subsample': 0.9,
        'colsample_bytree': 0.5,
        'seed': 42
    }

    # Train the model
    modelo_xgb = xgb.train(
        params,
        dtrain,
        num_boost_round=1000,
        evals=[(dtest, 'test')],
        early_stopping_rounds=10,
        verbose_eval=False
    )

    # Make predictions
    yhat_xgb = modelo_xgb.predict(dtest)
    yhat_xgb = yhat_xgb.argmax(axis=1)  # Convert probabilities to class predictions

    # Evaluate the model
    accu_xgb = accuracy_score(y_test-1, yhat_xgb)
    prec_xgb = precision_score(y_test-1, yhat_xgb, average='weighted')
    reca_xgb = recall_score(y_test-1, yhat_xgb, average='weighted')

    print('Accuracy\t Precision\t Recall')
    print(f'{accu_xgb:.3f}\t\t {prec_xgb:.3f}\t\t {reca_xgb:.3f}')

    # Log parameters
    mlflow.log_param("model_type", "XGBoost")
    mlflow.log_params(params)
    # Log metrics
    mlflow.log_metric("accuracy", accu_xgb)
    mlflow.log_metric("precision", prec_xgb)
    mlflow.log_metric("recall", reca_xgb)
    # Log model
    mlflow.xgboost.log_model(modelo_xgb, "model")
    # Get run ID
    xgb_run_id = mlflow.active_run().info.run_id

### Selección del mejor modelo y registro en MLflow

In [ ]:
# Comparar los modelos y seleccionar el mejor
data_compar_cross = {
    'Modelo': ['LR', 'SVC', 'RF', 'XGBoost'],
    'Accuracy': [acc_lr, accu_svc, accu_rf, accu_xgb],
    'Precision': [prec_lr, prec_svc, prec_rf, prec_xgb],
    'Recall': [rec_lr, reca_svc, reca_rf, reca_xgb]
}

data_compar_cross = pd.DataFrame(data_compar_cross)
print(data_compar_cross)

# Supongamos que XGBoost es el mejor modelo
best_run_id = xgb_run_id

# Registrar el mejor modelo en MLflow Model Registry
result = mlflow.register_model(f"runs:/{best_run_id}/model", "LaLigaBestModel")
print(f"Modelo registrado con nombre: {result.name}, versión: {result.version}")

In [ ]:
# Visualización de las métricas de los modelos
data_compar_cross.plot(x='Modelo', y=['Accuracy', 'Precision', 'Recall'], kind='bar')
plt.title('Comparación de Modelos')
plt.ylabel('Score')
plt.xticks(rotation=0)
plt.show()

### Fin del notebook con MLflow tracking